In [1]:
# Google的T5模型从2019年发布到今天雄风依旧；在翻译，文本分类，智能问答，文章摘要等方面都取得SOTA地位；本文使用T5的翻译功能完成 文本从一种语言翻译到另一种语言的翻译功能，我们可以使用把模型最后输出的目标语音文本代回到google翻译器中进行进一步验证，来判断T5模型的翻译功能效果。
# 一、目标文本是什么？

# 为了方便T5模型的训练，我们这里采用两句自己定义的英文文本，当然读者也可以自己定义相关句子；本文的目标文本为英语，目标文本为法语；
# 【注意：输入文本为list格式，元素为str格式；同时在task_prefix 定义翻译原文语言和目标语言】

#suppose we have the following 2 training examples
input_sequence_2 = "My eyes fill you with love"
input_sequence_3 = "He is pretty kind that I did not expect"
input_sequence_4 = "Her eyes are blue"


# 然后我们把它转化为list形式

task_prefix = "translate English to French: "
input_sequences = [input_sequence_2]
input_sequences.extend([input_sequence_3, input_sequence_4])

In [6]:
input_sequences

['My eyes fill you with love',
 'He is pretty kind that I did not expect',
 'Her eyes are blue']

In [2]:
# 二、模型调用步骤
# 1.引入库
# pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# 1.2 在python脚本中写入引入库的相关代码，如下

from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch


# 2.导入模型，本文使用 t5-base

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
#task specific parameter
max_source_length = 512

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes 

In [3]:
# 3.使用分词器对目标文本进行分词

encoding = tokenizer([task_prefix + sequence for sequence in input_sequences]
                    , padding = True
                    , return_tensors="pt").input_ids

In [4]:
encoding

tensor([[13959,  1566,    12,  2379,    10,   499,  2053,    14,    25,    28,
           333,     1,     0,     0,     0],
        [13959,  1566,    12,  2379,    10,   216,    19,  1134,   773,    24,
            27,   410,    59,  1672,     1],
        [13959,  1566,    12,  2379,    10,  1347,  2053,    33,  1692,     1,
             0,     0,     0,     0,     0]])

In [5]:
# 4.对刚刚生成的分词结果进行目标语言的生成工作

outputs = model.generate(encoding)


# 5.对生成的目标语言进行解码工作，就可得到目标语言的文本，并打印

result = tokenizer.batch_decode(outputs, skip_special_tokens = True)
print(result)

    
# 结果如下，

# ["Mes yeux vous remplissent d'amour", 'Il est assez gentil que je ne m’attendais pas à', 'Ses yeux sont bleus']


# 通过google检验：即再把法语翻译为英语，结果如下


# 完全一致！当然，因为法语是不分性别的，也就是他/她 不分，所以小伙伴们可能翻译的最终结果是 Her而不是His

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


["Mes yeux vous remplissent d'amour", 'Il est assez gentil que je ne m’attendais pas à', 'Ses yeux sont bleus']
